In [1]:
import torch

In [2]:
from simple_trainer import Config, Runner
import json
import os
import math

In [6]:
def read_json_and_make_config(config_file: str) -> Config:
    import json
    with open(config_file, 'r') as f:
        config = json.load(f)
    return Config(**config)

In [22]:
jsonstr1 = json.dumps(runner.cfg.__dict__, indent=2)

with open('cfg_test.json', 'w') as f:
    f.write(jsonstr1)

In [7]:
# read config stored in a json file
cfg= read_json_and_make_config("config.json")
runner = Runner(cfg)

#iterate through all attricutes of runner
for attr in dir(runner):
    if not attr.startswith("__"):
        print(attr, "---->", getattr(runner, attr))

[Parser] 50 images, taken by 10 cameras.
Scene scale: 1.4702541970170933
Model initialized. Number of GS: 60408
_viewer_render_fn ----> <bound method Runner._viewer_render_fn of <simple_trainer.Runner object at 0x7f4cd04bad00>>
app_optimizers ----> []
cfg ----> Config(disable_viewer=True, ckpt=None, resume=False, run=0, data_dir='/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/segmented_upright', data_factor=1, result_dir='/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/segmented_upright/results', test_every=8, patch_size=None, global_scale=1.0, port=8080, batch_size=1, steps_scaler=1.0, max_steps=30000, eval_steps=[7000, 30000], save_steps=[7000, 30000], init_type='sfm', init_num_pts=100000, init_extent=3.0, sh_degree=3, sh_degree_interval=1000, init_opa=0.1, init_scale=1.0, ssim_lambda=0.2, near_plane=0.01, far_plane=10000000000.0, prune_opa=0.005, grow_grad2d=0.0002, grow_scale3d=0.01, prune_scale3d=0.1, refine_start_iter=500, refine_stop_iter=15000, reset_ever

In [14]:
type(runner.splats)

torch.nn.modules.container.ParameterDict

In [17]:
ckpt = torch.load('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/segmented_upright/results/ckpts/ckpt_6999.pt')
# ckpt is a dictiorary with jsut one key 'splats' whose value is an OrderedDict
print(type(ckpt["splats"]))

params= ckpt["splats"]
splats= torch.nn.ParameterDict(params).to("cuda")
print(type(splats))

for key, value in splats.items():
    print(key, value.shape)

<class 'collections.OrderedDict'>
<class 'torch.nn.modules.container.ParameterDict'>
means3d torch.Size([95662, 3])
opacities torch.Size([95662])
quats torch.Size([95662, 4])
scales torch.Size([95662, 3])
sh0 torch.Size([95662, 1, 3])
shN torch.Size([95662, 15, 3])


In [16]:

batch_size: int = 1,
sparse_grad: bool = False,
device: str = "cuda"

# optimizers = [
#         (torch.optim.SparseAdam if sparse_grad else torch.optim.Adam)(
#             [{"params": splats[name].to(device), "lr": lr * math.sqrt(batch_size), "name": name}],
#             eps=1e-15 / math.sqrt(batch_size),
#             betas=(1 - batch_size * (1 - 0.9), 1 - batch_size * (1 - 0.999)),
#         )
#         for name, _, lr in params
    ]

ValueError: too many values to unpack (expected 3)

In [7]:
runner.eval(80085)

Running evaluation...


/cs/student/projects3/COMP0197/grp1/mini/envs/gaussian_splatting/lib/python3.8/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


PSNR: 21.958, SSIM: 0.8073, LPIPS: 0.271 Time: 0.449s/image Number of GS: 95662


# Custom progress and step with tqdm

In [31]:
# custom stating point and update in steps on tqdm bars
from tqdm import tqdm
import time, random

first_iter= 0
steps= 10
last_iter= steps*10

progress_bar = tqdm(range(first_iter, last_iter), desc='Training progress', )

for i in range(first_iter, last_iter):
    time.sleep(0.05)
    if i % 10 == 0:
        progress_bar.set_postfix({'loss': random.random(), 'acc': random.random()})
        progress_bar.update(steps)
progress_bar.close()

Training progress:  93%|█████████▎| 70/75 [00:03<00:00, 18.48it/s, loss=0.473, acc=0.785]
